In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
df = pd.read_csv("wowbgs2.csv", index_col=0)
df.head()

,Code,Faction,Class,KB,D,HK,DD,HD,Honor,Win,Lose,Rol,BE
Battleground,,,,,,,,,,,,,
WG,WG1,Horde,Hunter,1,3,14,48155,6641,532,1.0,NaN,dps,NaN
WG,WG1,Horde,Death Knight,1,3,12,27025,7106,377,1.0,NaN,dps,NaN
WG,WG1,Alliance,Paladin,0,1,19,824,93879,252,NaN,1.0,heal,NaN
WG,WG1,Alliance,Paladin,1,2,25,7046,98599,274,NaN,1.0,heal,NaN
WG,WG1,Alliance,Rogue,2,3,23,65483,19629,268,NaN,1.0,dps,NaN


In [4]:
df = df.drop('Lose',1)

In [5]:
df['Win'] = df['Win'].replace(np.NaN,0)
df['Win'] = df['Win'].replace(1.0,1)

In [6]:
def normalize(df,name,count = 1000):
    dd1 = df[name]
    print(name,' ',max(dd1)-min(dd1))
    dd = np.array([0]*len(dd1))
    
    for i in range(len(dd1)):
        dd[i]=  round((dd1[i]-min(dd1))/count)
    df[name] = dd
    return df

In [7]:
names = ['DD','HD','Honor']
for name in names:
    df = normalize(df,name,int((max(df[name])-min(df[name]))/100))

DD   246000
HD   325000
Honor   1217


In [14]:
nmaes = ['Code','Faction','Class','Rol']
for name in names:
    replaces = {list(set(df[name]))[i]: i for i in range(len(set(df[name])))}
    row = df[name]
    new_row = np.array([0]*len(row))
    for i in range(len(row)):
        new_row[i] = replaces[row[i]]
    print(new_row)
    df[name] = new_row

[20 11  0 ... 16 22  5]
[ 2  2 29 ...  3  6  6]
[43 30 20 ... 22 22 23]


In [15]:
df.head()

,Code,Faction,Class,KB,D,HK,DD,HD,Honor,Win,Rol,BE
Battleground,,,,,,,,,,,,
WG,WG1,Horde,Hunter,1,3,14,20,2,43,1.0,dps,NaN
WG,WG1,Horde,Death Knight,1,3,12,11,2,30,1.0,dps,NaN
WG,WG1,Alliance,Paladin,0,1,19,0,29,20,0.0,heal,NaN
WG,WG1,Alliance,Paladin,1,2,25,3,30,22,0.0,heal,NaN
WG,WG1,Alliance,Rogue,2,3,23,27,6,21,0.0,dps,NaN



<p>Большая часть классификатора написана. Нужно написать функцию predict.</p>


In [9]:
def make_dict(arr):
    dicta = {}
    for i in arr:
        if dicta.get(i)==None:
            dicta.update({i:1})
        else:
            dicta.update({i:dicta.get(i)+1})
    return dicta

In [10]:
from math import log
class NaiveBaies():    
    def fit(self, X, y):
        # Список классов
        for i in range(len(y)):
            y[i]=int(y[i])
        self.classes = np.unique(y)
        # Сохраним кол-во классов
        self.cls_counts = len(set(y))
        
        # Сохраним для каждого значения признака кол-во таких объектов (P(w_i|c)), чтобы потом было легче обучаться
        # self.feature_counts[cls][column_idx][i] - 1+ кол-во элементов в обучающей выборке, с классом cls, у которых
        # признак под номером column_idx равен i
        self.feature_counts = {}
        for cls in self.classes:
            cur_feature_counts = {}
            for j in range(X.shape[1]):
                # Добавим ко всем значениям единицу, чтобы не было нулей в вероятностях
                arr = X[y == cls][j]
                arr  = make_dict(arr)
                cur_feature_counts[j] = arr
            self.feature_counts[cls] = cur_feature_counts
        return self
        
    def predict(self, X_test):
        preds = []
        for i in range(len(X_test)):
            line = X_test[i]
            count = 0
            key = 0.0
            for cls in self.classes:
                ans = log(sum(self.feature_counts[cls]))
                for j in range(len(line)):
                    ans*= self.feature_counts.get(cls,1).get(j,1).get(line[j],1)
                if ans>=count:
                    count = ans
                    key = cls
            preds.append(key)
        return preds

In [11]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, random_state=42, shuffle=True)   


<p>Можно попробовать и логистическую регрессию.</p>


In [13]:
for train, test in kf.split(df):
    df_train = df.iloc[train]
    df_test = df.iloc[test]
    X_train, y_train = df_train.drop("Win", axis=1).values, df_train["Win"].values
    X_test, y_test = df_test.drop("Win", axis=1).values, df_test["Win"].values

    clf = NaiveBaies()
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    summ= 0
    for i in range (len(preds)):
        if preds[i]==y_test[i]:
            summ+=1
    print(summ/len(preds))

0.5111441307578009
0.45988112927191677
0.5297176820208024
0.5174721189591078


In [16]:
names = ['KB', 'D', 'HK', 'DD', 'HD', 'Honor', 'Win' ,'BE']
for name in names:
    df[name].fillna(int(df[name].mean()), inplace=True)

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

df = df.drop('Code',1)
df = df.drop('Faction',1)
df = df.drop('Class',1)
df = df.drop('Rol',1)

cross_val_score(LogisticRegression(), df.drop("Win", axis=1).values, df["Win"].values, scoring="accuracy", cv=4)

array([0.93387816, 0.91381872, 0.89598811, 0.8929368 ])